In [1]:
import os

import numpy as np

import torch
import torch.nn as nn

from torch.autograd import Variable

import torchvision.utils

from data_loader import get_loader
from make_gif import make_gif

from DiscrimiatorModule import Discriminator
from GeneratorModule import Generator

In [2]:
def init_weights(m) :
    name = type(m)

    if name == nn.Conv3d or name == nn.ConvTranspose2d or name == nn.ConvTranspose3d :
        m.weight.data.normal_(0.0, 0.01)
        m.bias.data.fill_(0)
    elif name == nn.BatchNorm2d or name == nn.BatchNorm3d :
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [3]:
pre_train = True

batch_size = 64
video_size = 64
epoch_size = 1000
        
#check GPU
is_gpu = torch.cuda.is_available()
print(is_gpu)

if is_gpu :
    dtype = torch.cuda.FloatTensor
else :
    dtype = torch.FloatTensor

if pre_train :
    D = torch.load('./model/D2.ckpt').type(dtype)
    G = torch.load('./model/G2.ckpt').type(dtype)
else :
    D = Discriminator()
    D = D.type(dtype)

    G = Generator()
    G = G.type(dtype)

    D.apply(init_weights)
    G.apply(init_weights)

criterion = nn.BCEWithLogitsLoss().type(dtype)

d_optimizer = torch.optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))

True


In [4]:
data_loader = get_loader(data_path='./dataset', image_size=video_size, batch_size=batch_size, num_workers=2)

for epoch in range(816, epoch_size + 1) :
    for iter, (video, y) in enumerate(data_loader) :
        local_batch_size = video.size()[0]
        
        real_labels = Variable(torch.ones(local_batch_size, 2).type(dtype))
        fake_labels = Variable(torch.zeros(local_batch_size, 2).type(dtype))
        
        # 1. Train Discriminator
        video_data = Variable(video).type(dtype)
        y = Variable(y).type(dtype)
        
        y_data = torch.unsqueeze(torch.unsqueeze(torch.unsqueeze(y, -1), -1), -1)
        y_data = y_data.expand(local_batch_size, 6, 32, 64, 64)
       
        
        
        # 1-1. Real Video
        outputs = D(video_data, y_data).view(local_batch_size, 2)
        d_loss_real = criterion(outputs, real_labels)

        
        
        # 1-2. Fake Video
        z = Variable(torch.randn(local_batch_size, 100) * 0.01).type(dtype)
        fake_videos = G(z, y)
        outputs = D(fake_videos, y_data).view(local_batch_size, 2)
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        
        D.zero_grad()
        G.zero_grad()
        d_loss.backward()
        d_optimizer.step()





        # 2. Train Generator
        z = Variable(torch.randn(local_batch_size, 100) * 0.01).type(dtype)
        fake_videos = G(z, y)
        outputs = D(fake_videos, y_data).view(local_batch_size, 2)

        g_loss = criterion(outputs, real_labels)
        D.zero_grad()
        G.zero_grad()

        g_loss.backward()
        g_optimizer.step()
          
        print('Epoch [%d/%d], Iter [%d/%d], d_loss: %.4f, g_loss: %.4f' % (epoch, epoch_size, iter, len(data_loader), d_loss.data[0], g_loss.data[0]))
    
    print('Model saving...')
    
    if epoch % 100 == 0 :
        torch.save(D, './model/D_' + str(epoch) + '.ckpt')
        torch.save(G, './model/G_' + str(epoch) + '.ckpt')
    else :  
        torch.save(D, './model/D2.ckpt')
        torch.save(G, './model/G2.ckpt')

Epoch [640/1000], Iter [0/49], d_loss: 0.4254, g_loss: 11.5233
Epoch [640/1000], Iter [1/49], d_loss: 0.0467, g_loss: 10.6913
Epoch [640/1000], Iter [2/49], d_loss: 0.0031, g_loss: 10.6660
Epoch [640/1000], Iter [3/49], d_loss: 0.0116, g_loss: 11.2399
Epoch [640/1000], Iter [4/49], d_loss: 0.0917, g_loss: 9.5956
Epoch [640/1000], Iter [5/49], d_loss: 0.0109, g_loss: 9.5206
Epoch [640/1000], Iter [6/49], d_loss: 0.0257, g_loss: 5.9944
Epoch [640/1000], Iter [7/49], d_loss: 0.0124, g_loss: 7.5848
Epoch [640/1000], Iter [8/49], d_loss: 0.0135, g_loss: 5.7578
Epoch [640/1000], Iter [9/49], d_loss: 0.0103, g_loss: 7.5100
Epoch [640/1000], Iter [10/49], d_loss: 0.0043, g_loss: 6.5364
Epoch [640/1000], Iter [11/49], d_loss: 0.0010, g_loss: 4.3390
Epoch [640/1000], Iter [12/49], d_loss: 0.0100, g_loss: 5.2261
Epoch [640/1000], Iter [13/49], d_loss: 0.1132, g_loss: 9.9010
Epoch [640/1000], Iter [14/49], d_loss: 0.0608, g_loss: 11.2917
Epoch [640/1000], Iter [15/49], d_loss: 0.0023, g_loss: 12.2

Epoch [642/1000], Iter [32/49], d_loss: 0.0217, g_loss: 5.7398
Epoch [642/1000], Iter [33/49], d_loss: 0.0284, g_loss: 6.9739
Epoch [642/1000], Iter [34/49], d_loss: 0.0352, g_loss: 5.5762
Epoch [642/1000], Iter [35/49], d_loss: 0.1092, g_loss: 4.6514
Epoch [642/1000], Iter [36/49], d_loss: 0.0135, g_loss: 3.8101
Epoch [642/1000], Iter [37/49], d_loss: 0.0456, g_loss: 4.1970
Epoch [642/1000], Iter [38/49], d_loss: 0.0485, g_loss: 6.4676
Epoch [642/1000], Iter [39/49], d_loss: 0.0567, g_loss: 7.1393
Epoch [642/1000], Iter [40/49], d_loss: 0.0837, g_loss: 5.3518
Epoch [642/1000], Iter [41/49], d_loss: 0.1533, g_loss: 2.5740
Epoch [642/1000], Iter [42/49], d_loss: 0.3541, g_loss: 11.2638
Epoch [642/1000], Iter [43/49], d_loss: 0.0298, g_loss: 14.2054
Epoch [642/1000], Iter [44/49], d_loss: 1.2253, g_loss: 5.2849
Epoch [642/1000], Iter [45/49], d_loss: 0.0351, g_loss: 1.8710
Epoch [642/1000], Iter [46/49], d_loss: 1.3821, g_loss: 12.9636
Epoch [642/1000], Iter [47/49], d_loss: 0.0003, g_lo

Epoch [645/1000], Iter [15/49], d_loss: 0.0068, g_loss: 6.3331
Epoch [645/1000], Iter [16/49], d_loss: 0.0028, g_loss: 6.9297
Epoch [645/1000], Iter [17/49], d_loss: 0.0048, g_loss: 3.7644
Epoch [645/1000], Iter [18/49], d_loss: 0.0103, g_loss: 6.7268
Epoch [645/1000], Iter [19/49], d_loss: 0.0039, g_loss: 5.3557
Epoch [645/1000], Iter [20/49], d_loss: 0.1151, g_loss: 9.6005
Epoch [645/1000], Iter [21/49], d_loss: 0.0026, g_loss: 10.6501
Epoch [645/1000], Iter [22/49], d_loss: 0.0216, g_loss: 9.2127
Epoch [645/1000], Iter [23/49], d_loss: 0.0739, g_loss: 8.2311
Epoch [645/1000], Iter [24/49], d_loss: 0.0085, g_loss: 5.9949
Epoch [645/1000], Iter [25/49], d_loss: 0.0049, g_loss: 5.9909
Epoch [645/1000], Iter [26/49], d_loss: 0.0092, g_loss: 7.0461
Epoch [645/1000], Iter [27/49], d_loss: 0.0187, g_loss: 5.2513
Epoch [645/1000], Iter [28/49], d_loss: 0.0150, g_loss: 4.7505
Epoch [645/1000], Iter [29/49], d_loss: 0.0734, g_loss: 6.2944
Epoch [645/1000], Iter [30/49], d_loss: 0.0390, g_loss

Epoch [647/1000], Iter [47/49], d_loss: 0.0068, g_loss: 5.3376
Epoch [647/1000], Iter [48/49], d_loss: 0.0022, g_loss: 12.4402
Model saving...
Epoch [648/1000], Iter [0/49], d_loss: 0.0103, g_loss: 7.6309
Epoch [648/1000], Iter [1/49], d_loss: 0.0044, g_loss: 6.0262
Epoch [648/1000], Iter [2/49], d_loss: 0.0140, g_loss: 5.8603
Epoch [648/1000], Iter [3/49], d_loss: 0.0090, g_loss: 6.1491
Epoch [648/1000], Iter [4/49], d_loss: 0.0387, g_loss: 6.4753
Epoch [648/1000], Iter [5/49], d_loss: 0.0148, g_loss: 5.8741
Epoch [648/1000], Iter [6/49], d_loss: 0.0136, g_loss: 5.9290
Epoch [648/1000], Iter [7/49], d_loss: 0.0699, g_loss: 4.5497
Epoch [648/1000], Iter [8/49], d_loss: 0.0103, g_loss: 4.9175
Epoch [648/1000], Iter [9/49], d_loss: 0.0469, g_loss: 5.0639
Epoch [648/1000], Iter [10/49], d_loss: 0.0209, g_loss: 5.8747
Epoch [648/1000], Iter [11/49], d_loss: 0.0134, g_loss: 6.2987
Epoch [648/1000], Iter [12/49], d_loss: 0.0046, g_loss: 6.4286
Epoch [648/1000], Iter [13/49], d_loss: 0.0060, 

Epoch [650/1000], Iter [30/49], d_loss: 0.0144, g_loss: 7.1818
Epoch [650/1000], Iter [31/49], d_loss: 0.0372, g_loss: 5.4492
Epoch [650/1000], Iter [32/49], d_loss: 0.0072, g_loss: 5.6247
Epoch [650/1000], Iter [33/49], d_loss: 0.0373, g_loss: 5.2967
Epoch [650/1000], Iter [34/49], d_loss: 0.0228, g_loss: 6.1709
Epoch [650/1000], Iter [35/49], d_loss: 0.0018, g_loss: 7.9264
Epoch [650/1000], Iter [36/49], d_loss: 0.1662, g_loss: 4.1996
Epoch [650/1000], Iter [37/49], d_loss: 0.0040, g_loss: 5.7952
Epoch [650/1000], Iter [38/49], d_loss: 0.3254, g_loss: 7.9133
Epoch [650/1000], Iter [39/49], d_loss: 0.0048, g_loss: 10.9756
Epoch [650/1000], Iter [40/49], d_loss: 0.0642, g_loss: 11.5957
Epoch [650/1000], Iter [41/49], d_loss: 0.2277, g_loss: 10.3848
Epoch [650/1000], Iter [42/49], d_loss: 0.0630, g_loss: 7.5678
Epoch [650/1000], Iter [43/49], d_loss: 0.0049, g_loss: 5.0659
Epoch [650/1000], Iter [44/49], d_loss: 0.0200, g_loss: 7.5835
Epoch [650/1000], Iter [45/49], d_loss: 0.0195, g_lo

...

Epoch [809/1000], Iter [38/49], d_loss: 0.0793, g_loss: 5.5507
Epoch [809/1000], Iter [39/49], d_loss: 0.0246, g_loss: 5.1697
Epoch [809/1000], Iter [40/49], d_loss: 0.0082, g_loss: 7.0006
Epoch [809/1000], Iter [41/49], d_loss: 0.0307, g_loss: 6.0756
Epoch [809/1000], Iter [42/49], d_loss: 0.0435, g_loss: 7.5349
Epoch [809/1000], Iter [43/49], d_loss: 0.0065, g_loss: 6.2982
Epoch [809/1000], Iter [44/49], d_loss: 0.0087, g_loss: 5.7502
Epoch [809/1000], Iter [45/49], d_loss: 0.0136, g_loss: 5.3164
Epoch [809/1000], Iter [46/49], d_loss: 0.0110, g_loss: 8.9767
Epoch [809/1000], Iter [47/49], d_loss: 0.0153, g_loss: 6.0367
Epoch [809/1000], Iter [48/49], d_loss: 0.1362, g_loss: 6.2121
Model saving...
Epoch [810/1000], Iter [0/49], d_loss: 0.1678, g_loss: 7.8915
Epoch [810/1000], Iter [1/49], d_loss: 0.0076, g_loss: 10.2565
Epoch [810/1000], Iter [2/49], d_loss: 0.0153, g_loss: 11.8186
Epoch [810/1000], Iter [3/49], d_loss: 0.0863, g_loss: 7.8385
Epoch [810/1000], Iter [4/49], d_loss: 0.

Epoch [812/1000], Iter [21/49], d_loss: 0.0186, g_loss: 8.0687
Epoch [812/1000], Iter [22/49], d_loss: 0.0020, g_loss: 8.1062
Epoch [812/1000], Iter [23/49], d_loss: 0.0112, g_loss: 7.2949
Epoch [812/1000], Iter [24/49], d_loss: 0.0039, g_loss: 7.7469
Epoch [812/1000], Iter [25/49], d_loss: 0.0341, g_loss: 7.3318
Epoch [812/1000], Iter [26/49], d_loss: 0.0059, g_loss: 6.9961
Epoch [812/1000], Iter [27/49], d_loss: 0.0092, g_loss: 4.2623
Epoch [812/1000], Iter [28/49], d_loss: 0.0124, g_loss: 7.5985
Epoch [812/1000], Iter [29/49], d_loss: 0.0106, g_loss: 8.4331
Epoch [812/1000], Iter [30/49], d_loss: 0.0080, g_loss: 6.6859
Epoch [812/1000], Iter [31/49], d_loss: 0.0257, g_loss: 6.5224
Epoch [812/1000], Iter [32/49], d_loss: 0.0052, g_loss: 7.8065
Epoch [812/1000], Iter [33/49], d_loss: 0.0008, g_loss: 8.2558
Epoch [812/1000], Iter [34/49], d_loss: 0.0015, g_loss: 7.2986
Epoch [812/1000], Iter [35/49], d_loss: 0.0013, g_loss: 6.2023
Epoch [812/1000], Iter [36/49], d_loss: 0.0060, g_loss:

Epoch [815/1000], Iter [4/49], d_loss: 0.0597, g_loss: 4.2929
Epoch [815/1000], Iter [5/49], d_loss: 0.0290, g_loss: 3.8768
Epoch [815/1000], Iter [6/49], d_loss: 0.0369, g_loss: 6.9391
Epoch [815/1000], Iter [7/49], d_loss: 0.0023, g_loss: 5.7564
Epoch [815/1000], Iter [8/49], d_loss: 0.0088, g_loss: 8.6614
Epoch [815/1000], Iter [9/49], d_loss: 0.0113, g_loss: 7.7223
Epoch [815/1000], Iter [10/49], d_loss: 0.0126, g_loss: 5.8872
Epoch [815/1000], Iter [11/49], d_loss: 0.0005, g_loss: 10.2553
Epoch [815/1000], Iter [12/49], d_loss: 0.0109, g_loss: 5.0707
Epoch [815/1000], Iter [13/49], d_loss: 0.0348, g_loss: 8.8802
Epoch [815/1000], Iter [14/49], d_loss: 0.0055, g_loss: 9.5070
Epoch [815/1000], Iter [15/49], d_loss: 0.0217, g_loss: 8.0648
Epoch [815/1000], Iter [16/49], d_loss: 0.0825, g_loss: 3.4953
Epoch [815/1000], Iter [17/49], d_loss: 0.0330, g_loss: 4.9427
Epoch [815/1000], Iter [18/49], d_loss: 0.1861, g_loss: 4.3306
Epoch [815/1000], Iter [19/49], d_loss: 0.0571, g_loss: 2.48

Process Process-354:
KeyboardInterrupt
Process Process-353:
Traceback (most recent call last):
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dskym0/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dskym0/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self

KeyboardInterrupt: 